# BERT 모델 구현
BERT의 기본 모델, BERT의 Masked Language Model 작업, 단어 벡터 표현의 확인을 구현합니다.


※ 이 장의 파일은 Ubuntu 환경에서의 동작을 전제로 하고 있습니다. Windows와 같이 문자 코드가 다른 환경에서는 동작에 주의하십시오.


# 8.2 학습 목표

1.	BERT의 Embeddings 모듈의 동작을 이해하고 구현할 수 있다
2.	BERT의 Self-Attention을 활용한 Transformer 부분인 BertLayer 모듈의 동작을 이해하고 구현할 수 있다
3.	BERT의 Pooler 모듈의 동작을 이해하고 구현할 수 있다



# 8.3 학습 목표

1.	BERT의 학습된 모델을 자신의 구현 모델에 로드할 수 있다
2.	BERT용의 단어 분할 클래스 등, 언어 데이터의 전처리 부분을 구현할 수 있다
3.	BERT로 단어 벡터를 꺼내 확인하는 내용을 구현할 수 있다



In [ ]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


In [ ]:
ROOT_PATH = '/gdrive/My Drive/Colab Notebooks/Lectures/091-093/'
data_dir = '{}{}'.format(ROOT_PATH, 'data/')
weights_dir = '{}{}'.format(ROOT_PATH, 'weights/')

In [ ]:
!pwd
%cd /gdrive/My\ Drive/Colab\ Notebooks/Lectures/091-093/
!ls

/content
/gdrive/My Drive/Colab Notebooks/Lectures/091-093
8-2-3_bert_base.ipynb  data				      utils  weights
8-4_bert_IMDb.ipynb    make_folders_and_data_downloads.ipynb  vocab


# 사전 준비

- 도서의 지시에 따라, 이 장에서 사용하는 데이터를 준비합니다
- pip install attrdict

위 명령으로 attrdict 패키지를 설치해 둡니다

In [ ]:
import math
import numpy as np

import torch
from torch import nn

# 8.2 BERT 구현

## BERT_Base 네트워크 설정 파일 읽기

In [ ]:
# 설정을 config.json에서 읽어들여, JSON 사전 변수를 오브젝트 변수로 변환
import json

config_file = '{}{}'.format(weights_dir, "bert_config.json")

# 파일을 열어 JSON으로 읽기
json_file = open(config_file, 'r')
config = json.load(json_file)

# 출력 확인
config

{'attention_probs_dropout_prob': 0.1,
 'hidden_act': 'gelu',
 'hidden_dropout_prob': 0.1,
 'hidden_size': 768,
 'initializer_range': 0.02,
 'intermediate_size': 3072,
 'max_position_embeddings': 512,
 'num_attention_heads': 12,
 'num_hidden_layers': 12,
 'type_vocab_size': 2,
 'vocab_size': 30522}

In [ ]:
# 이렇게 적는 것은 번거롭다...
config['hidden_size']


768

In [ ]:
!pip install attrdict

In [ ]:
# 사전 변수를 오브젝트 변수로
from attrdict import AttrDict

config = AttrDict(config)
config.hidden_size


768

## BERT용으로 LayerNormalization층을 정의

In [ ]:
# BERT용으로 LayerNormalization 층을 정의합니다.
# 세부 구현을 TensorFlow에 맞추고 있습니다.
class BertLayerNorm(nn.Module):
    """LayerNormalization층 """

    def __init__(self, hidden_size, eps=1e-12):
        super(BertLayerNorm, self).__init__()
        self.gamma = nn.Parameter(torch.ones(hidden_size))  # weight에 대한 것
        self.beta = nn.Parameter(torch.zeros(hidden_size))  # bias에 대한 것
        self.variance_epsilon = eps

    def forward(self, x):
        u = x.mean(-1, keepdim=True)
        s = (x - u).pow(2).mean(-1, keepdim=True)
        x = (x - u) / torch.sqrt(s + self.variance_epsilon)
        return self.gamma * x + self.beta


## Embeddings 모듈 구현

In [ ]:
# BERT의 Embeddings 모듈입니다
class BertEmbeddings(nn.Module):
    """문장의 단어 ID열과, 첫번째인지 두번째 문장인지의 정보를, 내장 벡터로 변환한다
    """

    def __init__(self, config):
        super(BertEmbeddings, self).__init__()

        # 3개의 벡터 표현 내장

        # Token Embedding: 단어 ID를 단어 벡터로 변환, 
        # vocab_size = 30522로, BERT의 학습된 모델에 사용된 vocabulary 양
        # hidden_size = 768로, 특징량 벡터의 길이는 768
        self.word_embeddings = nn.Embedding(
            config.vocab_size, config.hidden_size, padding_idx=0)
        # (주석)padding_idx=0은, idx=0의 단어 벡터는 0으로 한다. BERT의 vocabulary의 idx=0은 [PAD]임.

        # Transformer Positional Embedding: 위치 정보 텐서를 벡터로 변환
        # Transformer의 경우는 sin, cos로 이루어진 고정값이지만, BERT 학습시킴
        # max_position_embeddings = 512로, 문장의 길이는 512단어
        self.position_embeddings = nn.Embedding(
            config.max_position_embeddings, config.hidden_size)

        # Sentence Embedding: 첫번째, 두번째 문장을 벡터로 변환
        # type_vocab_size = 2
        self.token_type_embeddings = nn.Embedding(
            config.type_vocab_size, config.hidden_size)

        # 작성한 LayerNormalization층
        self.LayerNorm = BertLayerNorm(config.hidden_size, eps=1e-12)

        # Dropout　'hidden_dropout_prob': 0.1
        self.dropout = nn.Dropout(config.hidden_dropout_prob)

    def forward(self, input_ids, token_type_ids=None):
        '''
        input_ids:  [batch_size, seq_len] 문장의 단어 ID를 나열
        token_type_ids: [batch_size, seq_len] 각 단어가 1번째 문장인지, 2번째 문장인지를 나타내는 id
        '''

        # 1. Token Embeddings
        # 단어 ID를 단어 벡터로 변환
        words_embeddings = self.word_embeddings(input_ids)

        # 2. Sentence Embedding
        # token_type_ids가 없는 경우는 문장의 모든 단어를 첫번째 문장으로 하여, 0으로 설정
        # input_ids와 같은 크기의 제로 텐서를 작성
        if token_type_ids is None:
            token_type_ids = torch.zeros_like(input_ids)
        token_type_embeddings = self.token_type_embeddings(token_type_ids)

        # 3. Transformer Positional Embedding: 
        # [0, 1, 2 ・・・]로 문장의 길이만큼 숫자가 하나씩 올라간다
        # [batch_size, seq_len]의 텐서 position_ids를 작성
        # position_ids를 입력해서, position_embeddings 층에서 768차원의 텐서를 꺼낸다
        seq_length = input_ids.size(1)  # 문장의 길이
        position_ids = torch.arange(
            seq_length, dtype=torch.long, device=input_ids.device)
        position_ids = position_ids.unsqueeze(0).expand_as(input_ids)
        position_embeddings = self.position_embeddings(position_ids)

        # 3개의 내장 텐서를 더한다 [batch_size, seq_len, hidden_size]
        embeddings = words_embeddings + position_embeddings + token_type_embeddings

        # LayerNormalization과 Dropout을 실행
        embeddings = self.LayerNorm(embeddings)
        embeddings = self.dropout(embeddings)

        return embeddings


## BertLayer 모듈


In [ ]:
class BertLayer(nn.Module):
    '''BERT의 BertLayer 모듈입니다. Transformer가 됩니다'''

    def __init__(self, config):
        super(BertLayer, self).__init__()

        # Self-Attention 부분
        self.attention = BertAttention(config)

        # Self-Attention의 출력을 처리하는 전결합층
        self.intermediate = BertIntermediate(config)

        # Self-Attention에 의한 특징량과 BertLayer에 원래의 입력을 더하는 층
        self.output = BertOutput(config)

    def forward(self, hidden_states, attention_mask, attention_show_flg=False):
        '''
        hidden_states: Embedder 모듈의 출력 텐서 [batch_size, seq_len, hidden_size]
        attention_mask: Transformer의 마스크와 같은 기능의 마스킹
        attention_show_flg: Self-Attention의 가중치를 반환할지의 플래그
        '''
        if attention_show_flg == True:
            '''attention_show일 경우, attention_probs도 반환한다'''
            attention_output, attention_probs = self.attention(
                hidden_states, attention_mask, attention_show_flg)
            intermediate_output = self.intermediate(attention_output)
            layer_output = self.output(intermediate_output, attention_output)
            return layer_output, attention_probs

        elif attention_show_flg == False:
            attention_output = self.attention(
                hidden_states, attention_mask, attention_show_flg)
            intermediate_output = self.intermediate(attention_output)
            layer_output = self.output(intermediate_output, attention_output)

            return layer_output  # [batch_size, seq_length, hidden_size]


In [ ]:
class BertAttention(nn.Module):
    '''BertLayer 모듈의 Self-Attention 부분입니다'''
    def __init__(self, config):
        super(BertAttention, self).__init__()
        self.selfattn = BertSelfAttention(config)
        self.output = BertSelfOutput(config)

    def forward(self, input_tensor, attention_mask, attention_show_flg=False):
        '''
        input_tensor: Embeddings 모듈 또는 앞단의 BertLayer에서의 출력
        attention_mask: Transformer의 마스크와 같은 기능의 마스킹입니다
        attention_show_flg: Self-Attention의 가중치를 반환할지의 플래그
        '''
        if attention_show_flg == True:
            '''attention_show일 경우, attention_probs도 반환한다'''
            self_output, attention_probs = self.selfattn(input_tensor, attention_mask, attention_show_flg)
            attention_output = self.output(self_output, input_tensor)
            return attention_output, attention_probs
        
        elif attention_show_flg == False:
            self_output = self.selfattn(input_tensor, attention_mask, attention_show_flg)
            attention_output = self.output(self_output, input_tensor)
            return attention_output

In [ ]:
class BertSelfAttention(nn.Module):
    '''BertAttention의 Self-Attention입니다'''

    def __init__(self, config):
        super(BertSelfAttention, self).__init__()

        self.num_attention_heads = config.num_attention_heads
        # num_attention_heads': 12

        self.attention_head_size = int(
            config.hidden_size / config.num_attention_heads)  # 768/12=64
        self.all_head_size = self.num_attention_heads * \
            self.attention_head_size  # = 'hidden_size': 768

        # Self-Attention의 특징량을 작성하는 전결합층
        self.query = nn.Linear(config.hidden_size, self.all_head_size)
        self.key = nn.Linear(config.hidden_size, self.all_head_size)
        self.value = nn.Linear(config.hidden_size, self.all_head_size)

        # Dropout
        self.dropout = nn.Dropout(config.attention_probs_dropout_prob)

    def transpose_for_scores(self, x):
        '''multi-head Attention용으로 텐서의 형태를 변환한다
        [batch_size, seq_len, hidden] → [batch_size, 12, seq_len, hidden/12] 
        '''
        new_x_shape = x.size()[
            :-1] + (self.num_attention_heads, self.attention_head_size)
        x = x.view(*new_x_shape)
        return x.permute(0, 2, 1, 3)

    def forward(self, hidden_states, attention_mask, attention_show_flg=False):
        '''
        hidden_states: Embeddings 모듈 또는 앞단의 BertLayer에서의 출력
        attention_mask: Transformer의 마스크와 같은 기능의 마스킹입니다
        attention_show_flg: Self-Attention의 가중치를 반환할지의 플래그
        '''
        # 입력을 전결합층에서 특징량 변환(주의, multi-head Attention 전부를 한꺼번에 변환하고 있습니다)
        mixed_query_layer = self.query(hidden_states)
        mixed_key_layer = self.key(hidden_states)
        mixed_value_layer = self.value(hidden_states)

        # multi-head Attention용으로 텐서 형태를 변환
        query_layer = self.transpose_for_scores(mixed_query_layer)
        key_layer = self.transpose_for_scores(mixed_key_layer)
        value_layer = self.transpose_for_scores(mixed_value_layer)

        # 특징량끼리를 곱해서 비슷한 정도를 Attention_scores로 구한다
        attention_scores = torch.matmul(
            query_layer, key_layer.transpose(-1, -2))
        attention_scores = attention_scores / \
            math.sqrt(self.attention_head_size)

        # 마스크가 있는 부분에 마스크를 적용합니다
        attention_scores = attention_scores + attention_mask
        # (비고)
        # 마스크는 곱셈이 아니라 덧셈이 직관적이지만, 그 후에 Softmax로 정규화하므로,
        # 마스크된 부분은 -inf로 합니다. attention_mask에는, 0이나-inf가
        # 원래 들어 있으므로 덧셈으로 합니다.

        # Attention을 정규화한다
        attention_probs = nn.Softmax(dim=-1)(attention_scores)

        # 드롭아웃합니다
        attention_probs = self.dropout(attention_probs)

        # Attention Map을 곱합니다
        context_layer = torch.matmul(attention_probs, value_layer)

        # multi-head Attention의 텐서 형을 원래대로 되돌림
        context_layer = context_layer.permute(0, 2, 1, 3).contiguous()
        new_context_layer_shape = context_layer.size()[
            :-2] + (self.all_head_size,)
        context_layer = context_layer.view(*new_context_layer_shape)

        # attention_show일 경우, attention_probs도 반환한다
        if attention_show_flg == True:
            return context_layer, attention_probs
        elif attention_show_flg == False:
            return context_layer


In [ ]:
class BertSelfOutput(nn.Module):
    '''BertSelfAttention의 출력을 처리하는 전결합층입니다'''

    def __init__(self, config):
        super(BertSelfOutput, self).__init__()

        self.dense = nn.Linear(config.hidden_size, config.hidden_size)
        self.LayerNorm = BertLayerNorm(config.hidden_size, eps=1e-12)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        # 'hidden_dropout_prob': 0.1

    def forward(self, hidden_states, input_tensor):
        '''
        hidden_states: BertSelfAttention의 출력 텐서
        input_tensor: Embeddings 모듈 또는 앞단의 BertLayer에서의 출력
        '''
        hidden_states = self.dense(hidden_states)
        hidden_states = self.dropout(hidden_states)
        hidden_states = self.LayerNorm(hidden_states + input_tensor)
        return hidden_states


In [ ]:
def gelu(x):
    '''Gaussian Error Linear Unit라는 활성화 함수입니다.
    LeLU가 0으로 거칠고 불연속적이므로, 이를 연속적으로 매끄럽게 한 형태의 LeLU입니다.
    '''
    return x * 0.5 * (1.0 + torch.erf(x / math.sqrt(2.0)))


class BertIntermediate(nn.Module):
    '''BERT의 TransformerBlock 모듈의 FeedForward입니다'''
    def __init__(self, config):
        super(BertIntermediate, self).__init__()
        
        # 전결합층: 'hidden_size': 768, 'intermediate_size': 3072
        self.dense = nn.Linear(config.hidden_size, config.intermediate_size)
        
        # 활성화 함수 gelu
        self.intermediate_act_fn = gelu
            
    def forward(self, hidden_states):
        '''
        hidden_states:  BertAttention의 출력 텐서
        '''
        hidden_states = self.dense(hidden_states)
        hidden_states = self.intermediate_act_fn(hidden_states)  # GELU에 의한 활성화
        return hidden_states

In [ ]:
class BertOutput(nn.Module):
    '''BERT의 TransformerBlock 모듈의 FeedForward입니다'''

    def __init__(self, config):
        super(BertOutput, self).__init__()

        # 전결합층: 'intermediate_size': 3072, 'hidden_size': 768
        self.dense = nn.Linear(config.intermediate_size, config.hidden_size)

        self.LayerNorm = BertLayerNorm(config.hidden_size, eps=1e-12)

        # 'hidden_dropout_prob': 0.1
        self.dropout = nn.Dropout(config.hidden_dropout_prob)

    def forward(self, hidden_states, input_tensor):
        '''
        hidden_states:  BertIntermediate의 출력 텐서
        input_tensor: BertAttention의 출력 텐서
        '''
        hidden_states = self.dense(hidden_states)
        hidden_states = self.dropout(hidden_states)
        hidden_states = self.LayerNorm(hidden_states + input_tensor)
        return hidden_states


## BertLayer 모듈의 반복 부분

In [ ]:
# BertLayer 모듈의 반복 부분입니다
class BertEncoder(nn.Module):
    def __init__(self, config):
        '''BertLayer 모듈의 반복 부분입니다'''
        super(BertEncoder, self).__init__()

        # config.num_hidden_layers의 값. 즉, 12개의 BertLayer 모듈을 만듭니다
        self.layer = nn.ModuleList([BertLayer(config)
                                    for _ in range(config.num_hidden_layers)])

    def forward(self, hidden_states, attention_mask, output_all_encoded_layers=True, attention_show_flg=False):
        '''
        hidden_states: Embeddings 모듈의 출력
        attention_mask: Transformer의 마스크와 동일한 기능의 마스킹입니다
        output_all_encoded_layers: 반환 값을 전체 TransformerBlock 모듈의 출력으로 할지, 
        최후 층만으로 한정할지의 플래그.
        attention_show_flg: Self-Attention의 가중치를 반환할지의 플래그
        '''

        # 반환 값으로 사용할 리스트
        all_encoder_layers = []

        # BertLayer 모듈의 처리를 반복
        for layer_module in self.layer:

            if attention_show_flg == True:
                '''attention_show의 경우, attention_probs도 반환한다'''
                hidden_states, attention_probs = layer_module(
                    hidden_states, attention_mask, attention_show_flg)
            elif attention_show_flg == False:
                hidden_states = layer_module(
                    hidden_states, attention_mask, attention_show_flg)

            # 반환값으로 BertLayer에서 출력된 특징량을 12층 분, 모두 사용할 경우의 처리
            if output_all_encoded_layers:
                all_encoder_layers.append(hidden_states)

        # 반환값으로 최후의 BertLayer에서 출력된 특징량만을 사용할 경우의 처리
        if not output_all_encoded_layers:
            all_encoder_layers.append(hidden_states)

        # attention_show의 경우, attention_probs(마지막 12단)도 반환한다
        if attention_show_flg == True:
            return all_encoder_layers, attention_probs
        elif attention_show_flg == False:
            return all_encoder_layers


## BertPooler 모듈


In [ ]:
class BertPooler(nn.Module):
    '''입력 문장의 첫번째 단어[cls]의 특징량을 반환하고 유지하기 위한 모듈'''

    def __init__(self, config):
        super(BertPooler, self).__init__()

        # 전결합층, 'hidden_size': 768
        self.dense = nn.Linear(config.hidden_size, config.hidden_size)
        self.activation = nn.Tanh()

    def forward(self, hidden_states):
        # 1번째 단어의 특징량을 취득
        first_token_tensor = hidden_states[:, 0]

        # 전결합층에서 특징량 변환
        pooled_output = self.dense(first_token_tensor)

        # 활성화 함수 Tanh를 계산
        pooled_output = self.activation(pooled_output)

        return pooled_output


## 동작 확인

In [ ]:
# 동작 확인

# 입력 단어 ID열, batch_size는 두가지
input_ids = torch.LongTensor([[31, 51, 12, 23, 99], [15, 5, 1, 0, 0]])
print("입력 단어 ID열의 텐서 크기: ", input_ids.shape)

# 마스크
attention_mask = torch.LongTensor([[1, 1, 1, 1, 1], [1, 1, 1, 0, 0]])
print("입력 마스크의 텐서 크기: ", attention_mask.shape)

# 문장의 ID. 두 미니 배치 각각에 대해, 0은 첫번째 문장을, 1은 2번째 문장을 나타냄
token_type_ids = torch.LongTensor([[0, 0, 1, 1, 1], [0, 1, 1, 1, 1]])
print("입력 문장 ID의 텐서 크기: ", token_type_ids.shape)


# BERT의 각 모듈을 준비
embeddings = BertEmbeddings(config)
encoder = BertEncoder(config)
pooler = BertPooler(config)

# 마스크 변형　[batch_size, 1, 1, seq_length]으로 한다
# Attention을 적용하지 않는 부분은 마이너스 무한으로 하고 싶으므로, -10000을 곱하고 있습니다
extended_attention_mask = attention_mask.unsqueeze(1).unsqueeze(2)
extended_attention_mask = extended_attention_mask.to(dtype=torch.float32)
extended_attention_mask = (1.0 - extended_attention_mask) * -10000.0
print("확장된 마스크의 텐서 크기: ", extended_attention_mask.shape)

# 순전파한다
out1 = embeddings(input_ids, token_type_ids)
print("BertEmbeddings의 출력 텐서 크기: ", out1.shape)

out2 = encoder(out1, extended_attention_mask)
# out2는, [minibatch, seq_length, embedding_dim]가 12개의 리스트
print("BertEncoder 최후 층의 출력 텐서 크기: ", out2[0].shape)

out3 = pooler(out2[-1])  # out2는 12층의 특징량의 리스트가 되어 있으므로 가장 마지막을 사용
print("BertPooler의 출력 텐서 크기: ", out3.shape)


입력 단어 ID열의 텐서 크기:  torch.Size([2, 5])
입력 마스크의 텐서 크기:  torch.Size([2, 5])
입력 문장 ID의 텐서 크기:  torch.Size([2, 5])
확장된 마스크의 텐서 크기:  torch.Size([2, 1, 1, 5])
BertEmbeddings의 출력 텐서 크기:  torch.Size([2, 5, 768])
BertEncoder 최후 층의 출력 텐서 크기:  torch.Size([2, 5, 768])
BertPooler의 출력 텐서 크기:  torch.Size([2, 768])


## 전부 연결하여 BERT 모델로 

In [ ]:
class BertModel(nn.Module):
    '''모듈을 전부 연결한 BERT 모델'''

    def __init__(self, config):
        super(BertModel, self).__init__()

        # 3가지 모듈을 작성
        self.embeddings = BertEmbeddings(config)
        self.encoder = BertEncoder(config)
        self.pooler = BertPooler(config)

    def forward(self, input_ids, token_type_ids=None, attention_mask=None, output_all_encoded_layers=True, attention_show_flg=False):
        '''
        input_ids:  [batch_size, sequence_length] 문장의 단어 ID를 나열
        token_type_ids:  [batch_size, sequence_length] 각 단어가 1번째 문장인지, 2번째 문장인지를 나타내는 id
        attention_mask: Transformer의 마스크와 같은 기능의 마스킹
        output_all_encoded_layers: 최후 출력에 12단의 Transformer의 전부를 리스트로 반환할지, 최후만인지를 지정
        attention_show_flg: Self-Attention의 가중치를 반환할지의 플래그
        '''

        # Attention의 마스크와 첫번째, 두번째 문장의 id가 없으면 작성한다
        if attention_mask is None:
            attention_mask = torch.ones_like(input_ids)
        if token_type_ids is None:
            token_type_ids = torch.zeros_like(input_ids)

        # 마스크 변형 [minibatch, 1, 1, seq_length]으로 한다
        # 나중에 multi-head Attention에서 사용할 수 있는 형태로 하고 싶으므로
        extended_attention_mask = attention_mask.unsqueeze(1).unsqueeze(2)

        # 마스크는 0, 1이지만 소프트 맥스를 계산할 때 마스크가 되도록, 0과 -inf으로 한다
        # -inf 대신 -10000으로 해 둡니다
        extended_attention_mask = extended_attention_mask.to(
            dtype=torch.float32)
        extended_attention_mask = (1.0 - extended_attention_mask) * -10000.0

        # 순전파시킨다
        # BertEmbeddins 모듈
        embedding_output = self.embeddings(input_ids, token_type_ids)

        # BertLayer 모듈(Transformer)을 반복하는 BertEncoder 모듈
        if attention_show_flg == True:
            '''attention_show의 경우, attention_probs도 반환한다'''

            encoded_layers, attention_probs = self.encoder(embedding_output,
                                                           extended_attention_mask,
                                                           output_all_encoded_layers, attention_show_flg)

        elif attention_show_flg == False:
            encoded_layers = self.encoder(embedding_output,
                                          extended_attention_mask,
                                          output_all_encoded_layers, attention_show_flg)

        # BertPooler 모듈
        # encoder의 맨 마지막 BertLayer에서 출력된 특징량을 사용
        pooled_output = self.pooler(encoded_layers[-1])

        # output_all_encoded_layers가 False인 경우는 리스트가 아니라, 텐서를 반환
        if not output_all_encoded_layers:
            encoded_layers = encoded_layers[-1]

        # attention_show의 경우, attention_probs(가장 마지막)도 반환한다
        if attention_show_flg == True:
            return encoded_layers, pooled_output, attention_probs
        elif attention_show_flg == False:
            return encoded_layers, pooled_output


In [ ]:
# 동작 확인
# 입력을 준비
input_ids = torch.LongTensor([[31, 51, 12, 23, 99], [15, 5, 1, 0, 0]])
attention_mask = torch.LongTensor([[1, 1, 1, 1, 1], [1, 1, 1, 0, 0]])
token_type_ids = torch.LongTensor([[0, 0, 1, 1, 1], [0, 1, 1, 1, 1]])

# BERT 모델을 만들기
net = BertModel(config)

# 순전파시킨다
encoded_layers, pooled_output, attention_probs = net(
    input_ids, token_type_ids, attention_mask, output_all_encoded_layers=False, attention_show_flg=True)

print("encoded_layers의 텐서 크기: ", encoded_layers.shape)
print("pooled_output의 텐서 크기: ", pooled_output.shape)
print("attention_probs의 텐서 크기: ", attention_probs.shape)


encoded_layers의 텐서 크기:  torch.Size([2, 5, 768])
pooled_output의 텐서 크기:  torch.Size([2, 768])
attention_probs의 텐서 크기:  torch.Size([2, 12, 5, 5])


# 8.3 BERT를 이용한 bank(은행)과 bank(강변)의 단어 벡터 표현의 비교

## 학습된 모델을 로드

In [ ]:
# 학습된 모델을 로드
weights_path = "./weights/pytorch_model.bin"
loaded_state_dict = torch.load(weights_path)

for s in loaded_state_dict.keys():
    print(s, ': ', loaded_state_dict[s])


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
        -1.1873e-01,  1.8827e-02, -1.3362e-01,  1.6466e-01, -2.8234e-02,
        -1.2024e-01, -1.3766e-01,  1.3024e-01, -1.9420e-01,  1.1318e-01,
        -4.5297e-02, -5.0479e-02,  1.6549e-01, -3.7485e-02, -9.7978e-02,
        -1.6516e-01, -2.8699e-02,  1.7943e-02, -2.0056e-01, -9.9338e-02,
        -1.9311e-03, -5.4235e-02, -1.1880e-02,  2.3034e-02,  1.1718e-01,
        -1.4116e-01, -9.8754e-02,  2.1158e-02,  2.5026e-01, -1.0230e-01,
        -3.5182e-02, -2.7351e-01, -7.4305e-02,  2.4034e-01, -1.7391e-01,
         4.2071e-02,  1.1843e-01, -6.8441e-02,  6.3684e-02, -1.4631e-01,
         3.6316e-01,  6.8980e-02, -1.7664e-01,  5.1110e-02, -1.6003e-02,
        -2.5529e-01,  3.1410e-02,  1.7439e-01, -1.2248e-01, -9.4493e-02,
         2.1938e-02,  1.7721e-02, -1.4337e-01,  1.2564e-01, -1.6899e-01,
         6.0880e-03,  1.8392e-01, -2.8349e-01,  1.4458e-02, -7.3361e-02,
         7.2291e-03, -3.2562e-02, -6.2404e-02, -9.7708e-02, -2.6319e-02,
        -1.7973

In [ ]:
# 모델 준비
net = BertModel(config)
net.eval()

# 현재 네트워크 모델의 파라미터 이름
param_names = []  # 파라미터 이름을 저장해 나간다

for name, param in net.named_parameters():
    print(name)
    param_names.append(name)


embeddings.word_embeddings.weight
embeddings.position_embeddings.weight
embeddings.token_type_embeddings.weight
embeddings.LayerNorm.gamma
embeddings.LayerNorm.beta
encoder.layer.0.attention.selfattn.query.weight
encoder.layer.0.attention.selfattn.query.bias
encoder.layer.0.attention.selfattn.key.weight
encoder.layer.0.attention.selfattn.key.bias
encoder.layer.0.attention.selfattn.value.weight
encoder.layer.0.attention.selfattn.value.bias
encoder.layer.0.attention.output.dense.weight
encoder.layer.0.attention.output.dense.bias
encoder.layer.0.attention.output.LayerNorm.gamma
encoder.layer.0.attention.output.LayerNorm.beta
encoder.layer.0.intermediate.dense.weight
encoder.layer.0.intermediate.dense.bias
encoder.layer.0.output.dense.weight
encoder.layer.0.output.dense.bias
encoder.layer.0.output.LayerNorm.gamma
encoder.layer.0.output.LayerNorm.beta
encoder.layer.1.attention.selfattn.query.weight
encoder.layer.1.attention.selfattn.query.bias
encoder.layer.1.attention.selfattn.key.weight
e

In [ ]:
# state_dict의 이름이 다르므로 앞쪽부터 순서대로 대입한다
# 이번에는 파라미터 이름이 달라도, 대응하는 것은 동일한 순서로 되어 있습니다

# 현재 네트워크 정보를 복사하여 새로운 state_dict을 작성
new_state_dict = net.state_dict().copy()

# 새로운 state_dict에 학습된 값을 대입
for index, (key_name, value) in enumerate(loaded_state_dict.items()):
    name = param_names[index]  # 현재 네트워크의 파라미터명을 취득
    new_state_dict[name] = value  # 값을 넣는다
    print(str(key_name)+"→"+str(name))  # 어디로 들어갔는지 표시

    # 현재 네트워크의 파라미터를 전부 로드하면 끝낸다
    if index+1 >= len(param_names):
        break

# 새로운 state_dict를 구축한 BERT 모델에 제공
net.load_state_dict(new_state_dict)


bert.embeddings.word_embeddings.weight→embeddings.word_embeddings.weight
bert.embeddings.position_embeddings.weight→embeddings.position_embeddings.weight
bert.embeddings.token_type_embeddings.weight→embeddings.token_type_embeddings.weight
bert.embeddings.LayerNorm.gamma→embeddings.LayerNorm.gamma
bert.embeddings.LayerNorm.beta→embeddings.LayerNorm.beta
bert.encoder.layer.0.attention.self.query.weight→encoder.layer.0.attention.selfattn.query.weight
bert.encoder.layer.0.attention.self.query.bias→encoder.layer.0.attention.selfattn.query.bias
bert.encoder.layer.0.attention.self.key.weight→encoder.layer.0.attention.selfattn.key.weight
bert.encoder.layer.0.attention.self.key.bias→encoder.layer.0.attention.selfattn.key.bias
bert.encoder.layer.0.attention.self.value.weight→encoder.layer.0.attention.selfattn.value.weight
bert.encoder.layer.0.attention.self.value.bias→encoder.layer.0.attention.selfattn.value.bias
bert.encoder.layer.0.attention.output.dense.weight→encoder.layer.0.attention.output

<All keys matched successfully>

## BERT용 Tokenizer 구현

In [ ]:
# vocab 파일을 읽기
import collections

def load_vocab(vocab_file):
    """text 형식의 vocab 파일의 내용을 사전에 저장합니다"""
    vocab = collections.OrderedDict()  # (단어, id) 순서의 사전 변수
    ids_to_tokens = collections.OrderedDict()  # (id, 단어) 순서의 사전 변수
    index = 0

    with open(vocab_file, "r", encoding="utf-8") as reader:
        while True:
            token = reader.readline()
            if not token:
                break
            token = token.strip()

            # 저장
            vocab[token] = index
            ids_to_tokens[index] = token
            index += 1

    return vocab, ids_to_tokens


# 실행
vocab_file = "./vocab/bert-base-uncased-vocab.txt"
vocab, ids_to_tokens = load_vocab(vocab_file)


In [ ]:
vocab

OrderedDict([('[PAD]', 0),
             ('[unused0]', 1),
             ('[unused1]', 2),
             ('[unused2]', 3),
             ('[unused3]', 4),
             ('[unused4]', 5),
             ('[unused5]', 6),
             ('[unused6]', 7),
             ('[unused7]', 8),
             ('[unused8]', 9),
             ('[unused9]', 10),
             ('[unused10]', 11),
             ('[unused11]', 12),
             ('[unused12]', 13),
             ('[unused13]', 14),
             ('[unused14]', 15),
             ('[unused15]', 16),
             ('[unused16]', 17),
             ('[unused17]', 18),
             ('[unused18]', 19),
             ('[unused19]', 20),
             ('[unused20]', 21),
             ('[unused21]', 22),
             ('[unused22]', 23),
             ('[unused23]', 24),
             ('[unused24]', 25),
             ('[unused25]', 26),
             ('[unused26]', 27),
             ('[unused27]', 28),
             ('[unused28]', 29),
             ('[unused29]', 30),
  

In [ ]:
ids_to_tokens

OrderedDict([(0, '[PAD]'),
             (1, '[unused0]'),
             (2, '[unused1]'),
             (3, '[unused2]'),
             (4, '[unused3]'),
             (5, '[unused4]'),
             (6, '[unused5]'),
             (7, '[unused6]'),
             (8, '[unused7]'),
             (9, '[unused8]'),
             (10, '[unused9]'),
             (11, '[unused10]'),
             (12, '[unused11]'),
             (13, '[unused12]'),
             (14, '[unused13]'),
             (15, '[unused14]'),
             (16, '[unused15]'),
             (17, '[unused16]'),
             (18, '[unused17]'),
             (19, '[unused18]'),
             (20, '[unused19]'),
             (21, '[unused20]'),
             (22, '[unused21]'),
             (23, '[unused22]'),
             (24, '[unused23]'),
             (25, '[unused24]'),
             (26, '[unused25]'),
             (27, '[unused26]'),
             (28, '[unused27]'),
             (29, '[unused28]'),
             (30, '[unused29]'),
  

In [ ]:
from utils.tokenizer import BasicTokenizer, WordpieceTokenizer

# BasicTokenizer, WordpieceTokenizer는, 참고 문헌[2] 그대로입니다
# https://github.com/huggingface/pytorch-pretrained-BERT/blob/master/pytorch_pretrained_bert/tokenization.py
# sub-word로 단어 분할을 실시하는 클래스들입니다.
class BertTokenizer(object):
    '''BERT용의 문장 단어 분할 클래스를 구현'''

    def __init__(self, vocab_file, do_lower_case=True):
        '''
        vocab_file: vocabulary에의 경로
        do_lower_case: 전처리에서 단어를 소문자로 바꾸는지 여부
        '''

        # vocabulary의 로드
        self.vocab, self.ids_to_tokens = load_vocab(vocab_file)

        # 분할 처리 함수를 "utils" 폴더에서 imoprt, sub-word로 단어 분할을 실시
        never_split = ("[UNK]", "[SEP]", "[PAD]", "[CLS]", "[MASK]")
        # (주석)위 단어는 도중에 분할하지 않는다. 이를 통해 하나의 단어로 간주함

        self.basic_tokenizer = BasicTokenizer(do_lower_case=do_lower_case,
                                              never_split=never_split)
        self.wordpiece_tokenizer = WordpieceTokenizer(vocab=self.vocab)

    def tokenize(self, text):
        '''문장의 단어를 분할하는 함수'''
        split_tokens = []  # 분할 후 단어들
        for token in self.basic_tokenizer.tokenize(text):
            for sub_token in self.wordpiece_tokenizer.tokenize(token):
                split_tokens.append(sub_token)
        return split_tokens

    def convert_tokens_to_ids(self, tokens):
        """분할된 단어 목록을 ID로 변환하는 함수"""
        ids = []
        for token in tokens:
            ids.append(self.vocab[token])

        return ids

    def convert_ids_to_tokens(self, ids):
        """ID를 단어로 변환하는 함수"""
        tokens = []
        for i in ids:
            tokens.append(self.ids_to_tokens[i])
        return tokens


## Bank의 문맥에 따른 의미 변화를 단어 벡터로 구하기

In [ ]:
# 문장1: 은행 계좌에 접근했습니다.
text_1 = "[CLS] I accessed the bank account. [SEP]"

# 문장2: 그는 보증금을 은행 계좌로 이체했습니다.
text_2 = "[CLS] He transferred the deposit money into the bank account. [SEP]"

# 문장3: 우리는 강변에서 축구를 합니다.
text_3 = "[CLS] We play soccer at the bank of the river. [SEP]"

# 단어 분할 Tokenizer를 준비
tokenizer = BertTokenizer(
    vocab_file="./vocab/bert-base-uncased-vocab.txt", do_lower_case=True)

# 문장의 단어를 분할
tokenized_text_1 = tokenizer.tokenize(text_1)
tokenized_text_2 = tokenizer.tokenize(text_2)
tokenized_text_3 = tokenizer.tokenize(text_3)

# 확인
print(tokenized_text_1)
print(tokenized_text_2)
print(tokenized_text_3)

['[CLS]', 'i', 'accessed', 'the', 'bank', 'account', '.', '[SEP]']
['[CLS]', 'he', 'transferred', 'the', 'deposit', 'money', 'into', 'the', 'bank', 'account', '.', '[SEP]']
['[CLS]', 'we', 'play', 'soccer', 'at', 'the', 'bank', 'of', 'the', 'river', '.', '[SEP]']


In [ ]:
# 단어를 ID로 변환하기
indexed_tokens_1 = tokenizer.convert_tokens_to_ids(tokenized_text_1)
indexed_tokens_2 = tokenizer.convert_tokens_to_ids(tokenized_text_2)
indexed_tokens_3 = tokenizer.convert_tokens_to_ids(tokenized_text_3)

# 각 문장의 bank 위치
bank_posi_1 = np.where(np.array(tokenized_text_1) == "bank")[0][0]  # 4
bank_posi_2 = np.where(np.array(tokenized_text_2) == "bank")[0][0]  # 8
bank_posi_3 = np.where(np.array(tokenized_text_3) == "bank")[0][0]  # 6

# seqId(첫번째인지 2번째 문장인지는 이번에는 필요 없음)

# 리스트를 PyTorch의 텐서로
tokens_tensor_1 = torch.tensor([indexed_tokens_1])
tokens_tensor_2 = torch.tensor([indexed_tokens_2])
tokens_tensor_3 = torch.tensor([indexed_tokens_3])

# bank의 단어 id
bank_word_id = tokenizer.convert_tokens_to_ids(["bank"])[0]

# 확인
print(tokens_tensor_1)


tensor([[  101,  1045, 11570,  1996,  2924,  4070,  1012,   102]])


In [ ]:
# 문장을 BERT로 처리
with torch.no_grad():
    encoded_layers_1, _ = net(tokens_tensor_1, output_all_encoded_layers=True)
    encoded_layers_2, _ = net(tokens_tensor_2, output_all_encoded_layers=True)
    encoded_layers_3, _ = net(tokens_tensor_3, output_all_encoded_layers=True)


In [ ]:
# bank의 초기 단어 벡터 표현
# 이것은 Embeddings 모듈에서 꺼내, 단어 bank의 id에 따른 단어 벡터이므로 세 문장에서 공통
bank_vector_0 = net.embeddings.word_embeddings.weight[bank_word_id]

# 문장1의 BertLayer 모듈 1단에서 출력되는 bank의 특징량 벡터
bank_vector_1_1 = encoded_layers_1[0][0, bank_posi_1]

# 문장1의 BertLayer 모듈 최후 12단에서 출력되는 bank의 특징량 벡터
bank_vector_1_12 = encoded_layers_1[11][0, bank_posi_1]

# 문장2, 3도 마찬가지로
bank_vector_2_1 = encoded_layers_2[0][0, bank_posi_2]
bank_vector_2_12 = encoded_layers_2[11][0, bank_posi_2]
bank_vector_3_1 = encoded_layers_3[0][0, bank_posi_3]
bank_vector_3_12 = encoded_layers_3[11][0, bank_posi_3]


In [ ]:
# 코사인 유사도를 계산
import torch.nn.functional as F

print("bank의 초기 벡터와 문장1의 1단 bank의 유사도: ",
      F.cosine_similarity(bank_vector_0, bank_vector_1_1, dim=0))
print("bank의 초기 벡터와 문장1의 12단 bank의 유사도: ",
      F.cosine_similarity(bank_vector_0, bank_vector_1_12, dim=0))

print("문장1의 1층 bank와 문장2의 1단 bank의 유사도: ",
      F.cosine_similarity(bank_vector_1_1, bank_vector_2_1, dim=0))
print("문장1의 1층 bank와 문장3의 1단 bank의 유사도: ",
      F.cosine_similarity(bank_vector_1_1, bank_vector_3_1, dim=0))

print("문장1의 12층 bank와 문장2의 12단 bank의 유사도: ",
      F.cosine_similarity(bank_vector_1_12, bank_vector_2_12, dim=0))
print("문장1의 12층 bank와 문장3의 12단 bank의 유사도: ",
      F.cosine_similarity(bank_vector_1_12, bank_vector_3_12, dim=0))


bank의 초기 벡터와 문장1의 1단 bank의 유사도:  tensor(0.6814, grad_fn=<DivBackward0>)
bank의 초기 벡터와 문장1의 12단 bank의 유사도:  tensor(0.2276, grad_fn=<DivBackward0>)
문장1의 1층 bank와 문장2의 1단 bank의 유사도:  tensor(0.8968)
문장1의 1층 bank와 문장3의 1단 bank의 유사도:  tensor(0.7584)
문장1의 12층 bank와 문장2의 12단 bank의 유사도:  tensor(0.8796)
문장1의 12층 bank와 문장3의 12단 bank의 유사도:  tensor(0.4814)


여기까지의 내용을 "utils" 폴더의 bert.py에 별도로 저장해두고, 다음 절부터는 이를 로드합니다

# 부록

## 사전 학습 과제용 모듈을 구현

In [ ]:
class BertPreTrainingHeads(nn.Module):
    '''BERT의 사전 학습 과제를 수행하는 어댑터 모듈'''

    def __init__(self, config, bert_model_embedding_weights):
        super(BertPreTrainingHeads, self).__init__()

        # 사전 학습 과제: Masked Language Model용 모듈
        self.predictions = MaskedWordPredictions(config)

        # 사전 학습 과제: Next Sentence Prediction용 모듈
        self.seq_relationship = nn.Linear(config.hidden_size, 2)

    def forward(self, sequence_output, pooled_output):
        '''입력 정보
        sequence_output:[batch_size, seq_len, hidden_size]
        pooled_output:[batch_size, hidden_size]
        '''
        # 입력의 마스크된 각 단어가 어떤 단어인지를 판정
        # 출력 [minibatch, seq_len, vocab_size]
        prediction_scores = self.predictions(sequence_output)

        # 선두 단어의 특징량에서 1번째, 2번째 문장이 연결되어 있는지 판정
        seq_relationship_score = self.seq_relationship(
            pooled_output)  # 출력 [minibatch, 2]

        return prediction_scores, seq_relationship_score


In [ ]:
class BertPreTrainingHeads(nn.Module):
    def __init__(self, config):
        '''BERT의 사전 학습 과제를 수행하는 어댑터 모듈'''
        super(BertPreTrainingHeads, self).__init__()

        # 사전 학습 과제: Masked Language Model용 모듈
        self.predictions = MaskedWordPredictions(config)

        # 사전 학습 과제: Next Sentence Prediction용 모듈
        self.seq_relationship = SeqRelationship(config, out_features=2)

    def forward(self, sequence_output, pooled_output):
        '''입력 정보
        sequence_output:[batch_size, seq_len, hidden_size]
        pooled_output:[batch_size, hidden_size]
        '''
        # 입력의 마스크된 각 단어가 어떤 단어인지를 판정
        # 출력 [batch_size, seq_len, hidden_size]
        prediction_scores = self.predictions(sequence_output)

        # 선두 단어의 특징량에서 1번째, 2번째 문장이 연결되어 있는지 판정
        seq_relationship_score = self.seq_relationship(
            pooled_output)  # 출력 [batch_size, 2]

        return prediction_scores, seq_relationship_score


In [ ]:
# 사전 학습 과제: Masked Language Model용 모듈
class MaskedWordPredictions(nn.Module):
    def __init__(self, config):
        '''사전 학습 과제: Masked Language Model용 모듈
        원래의 [2] 구현에서는, BertLMPredictionHead이라는 이름입니다.
        '''
        super(MaskedWordPredictions, self).__init__()

        # BERT에서 출력된 특징량을 변환하는 모듈(입출력 크기는 동일)
        self.transform = BertPredictionHeadTransform(config)

        # self.transform의 출력에서, 각 위치의 단어가 어떤 것인지를 알아맞히는 전결합층
        self.decoder = nn.Linear(in_features=config.hidden_size,  # 'hidden_size': 768
                                 out_features=config.vocab_size,  # 'vocab_size': 30522
                                 bias=False)
        # 바이어스 항
        self.bias = nn.Parameter(torch.zeros(
            config.vocab_size))  # 'vocab_size': 30522

    def forward(self, hidden_states):
        '''
        hidden_states: BERT에서의 출력[batch_size, seq_len, hidden_size]
        '''
        # BERT에서 출력된 특징량을 변환
        # 출력 크기: [batch_size, seq_len, hidden_size]
        hidden_states = self.transform(hidden_states)

        # 각 위치의 단어가 vocabulary의 어느 단어인지를 클래스 분류로 예측
        # 출력 크기: [batch_size, seq_len, vocab_size]
        hidden_states = self.decoder(hidden_states) + self.bias

        return hidden_states


class BertPredictionHeadTransform(nn.Module):
    '''MaskedWordPredictions에서, BERT의 특징량을 변환하는 모듈(입출력 크기는 동일)'''

    def __init__(self, config):
        super(BertPredictionHeadTransform, self).__init__()

        # 전결합층 'hidden_size': 768
        self.dense = nn.Linear(config.hidden_size, config.hidden_size)

        # 활성화 함수 gelu
        self.transform_act_fn = gelu

        # LayerNormalization
        self.LayerNorm = BertLayerNorm(config.hidden_size, eps=1e-12)

    def forward(self, hidden_states):
        '''hidden_statesはsequence_output:[minibatch, seq_len, hidden_size]'''
        # 전결합층에서 특징량 변환하여, 활성화 함수 gelu를 게산한 뒤, LayerNormalization을 수행
        hidden_states = self.dense(hidden_states)
        hidden_states = self.transform_act_fn(hidden_states)
        hidden_states = self.LayerNorm(hidden_states)
        return hidden_states


In [ ]:
# 사전 학습 과제: Next Sentence Prediction용 모듈
class SeqRelationship(nn.Module):
    def __init__(self, config, out_features):
        '''사전 학습 과제: Next Sentence Prediction용 모듈
        원래 인수[2] 구현에서는, 특별히 크래스로 제공하고 있지 않음.
        일반적인 전결합층에, 일부러 이름을 붙임.
        '''
        super(SeqRelationship, self).__init__()

        # 선두 단어의 특징량에서 1번째, 2번째 문장이 연결되어 있는지 판정하는 클래스 분류의 전결합층
        self.seq_relationship = nn.Linear(config.hidden_size, out_features)

    def forward(self, pooled_output):
        return self.seq_relationship(pooled_output)


In [ ]:
class BertForMaskedLM(nn.Module):
    '''BERT 모델에 사전 학습 과제용의 어댑터 모듈
    BertPreTrainingHeads를 연결한 모델'''

    def __init__(self, config, net_bert):
        super(BertForMaskedLM, self).__init__()

        # BERT 모듈
        self.bert = net_bert

        # 사전 학습 과제용 어댑터 모듈
        self.cls = BertPreTrainingHeads(config)

    def forward(self, input_ids, token_type_ids=None, attention_mask=None):
        '''
        input_ids:  [batch_size, sequence_length] 문장의 단어 ID를 나열
        token_type_ids:  [batch_size, sequence_length] 각 단어가 1번째 문장인지, 2번째 문장인지를 나타내는 id
        attention_mask: Transformer의 마스크와 같은 기능의 마스킹
        '''

        # BERT의 기본 모델 부분의 순전파
        encoded_layers, pooled_output = self.bert(
            input_ids, token_type_ids, attention_mask, output_all_encoded_layers=False, attention_show_flg=False)

        # 사전 학습 과제의 추론을 실시
        prediction_scores, seq_relationship_score = self.cls(
            encoded_layers, pooled_output)

        return prediction_scores, seq_relationship_score


## 학습된 모델의 로드 부분을 구현합니다

In [ ]:
# BERT의 기본 모델
net_bert = BertModel(config)
net_bert.eval()

# 사전 학습 과제의 어댑터 모듈을 탑재한 BERT
net = BertForMaskedLM(config, net_bert)
net.eval()

# 학습된 가중치를 로드
weights_path = "./weights/pytorch_model.bin"
loaded_state_dict = torch.load(weights_path)


# 현재 네트워크 모델의 파라미터명
param_names = []  # 파라미터 이름을 저장해 나간다

for name, param in net.named_parameters():
    param_names.append(name)


# 현재 네트워크 정보를 복사하여 새로운 state_dict를 작성
new_state_dict = net.state_dict().copy()

# 새로운 state_dict에 학습된 값을 대입
for index, (key_name, value) in enumerate(loaded_state_dict.items()):
    name = param_names[index]  # 현재 네트워크의 파라미터명을 취득
    new_state_dict[name] = value  # 값을 넣는다
    print(str(key_name)+"→"+str(name))  # 어디로 들어갔는지 표시

    # 현재 네트워크의 파라미터를 전부 로드하면 끝낸다
    if index+1 >= len(param_names):
        break

# 새로운 state_dict를 구축한 BERT 모델에 제공
net.load_state_dict(new_state_dict)


bert.embeddings.word_embeddings.weight→bert.embeddings.word_embeddings.weight
bert.embeddings.position_embeddings.weight→bert.embeddings.position_embeddings.weight
bert.embeddings.token_type_embeddings.weight→bert.embeddings.token_type_embeddings.weight
bert.embeddings.LayerNorm.gamma→bert.embeddings.LayerNorm.gamma
bert.embeddings.LayerNorm.beta→bert.embeddings.LayerNorm.beta
bert.encoder.layer.0.attention.self.query.weight→bert.encoder.layer.0.attention.selfattn.query.weight
bert.encoder.layer.0.attention.self.query.bias→bert.encoder.layer.0.attention.selfattn.query.bias
bert.encoder.layer.0.attention.self.key.weight→bert.encoder.layer.0.attention.selfattn.key.weight
bert.encoder.layer.0.attention.self.key.bias→bert.encoder.layer.0.attention.selfattn.key.bias
bert.encoder.layer.0.attention.self.value.weight→bert.encoder.layer.0.attention.selfattn.value.weight
bert.encoder.layer.0.attention.self.value.bias→bert.encoder.layer.0.attention.selfattn.value.bias
bert.encoder.layer.0.attenti

<All keys matched successfully>

## 사전 학습 과제 Masked Language Model을 시험하기

In [ ]:
# 입력할 문장을 준비
text = "[CLS] I accessed the bank account. [SEP] We play soccer at the bank of the river. [SEP]"

# 단어 분할 Tokenizer을 준비
tokenizer = BertTokenizer(
    vocab_file="./vocab/bert-base-uncased-vocab.txt", do_lower_case=True)

# 문장의 단어를 분할
tokenized_text = tokenizer.tokenize(text)

print(tokenized_text)


['[CLS]', 'i', 'accessed', 'the', 'bank', 'account', '.', '[SEP]', 'we', 'play', 'soccer', 'at', 'the', 'bank', 'of', 'the', 'river', '.', '[SEP]']


In [ ]:
# 단어를 마스크한다. 이번에는 13번째 단어의 bank를 마스크
masked_index = 13
tokenized_text[masked_index] = '[MASK]'

print(tokenized_text)  # 13번째 단어가 [MASK] 되어 있다


['[CLS]', 'i', 'accessed', 'the', 'bank', 'account', '.', '[SEP]', 'we', 'play', 'soccer', 'at', 'the', '[MASK]', 'of', 'the', 'river', '.', '[SEP]']


In [ ]:
# 단어를 ID로 변환한다
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
print(indexed_tokens)


[101, 1045, 11570, 1996, 2924, 4070, 1012, 102, 2057, 2377, 4715, 2012, 1996, 103, 1997, 1996, 2314, 1012, 102]


In [ ]:
# 1번째 문장에 0을, 2번째 문장에 1을 넣어 문장 ID를 준비
def seq2id(indexed_tokens):
    '''띄어쓰기된 단어 ID열을 문장 ID로. [SEP]으로 나누기'''

    segments_ids = []
    seq_id = 0

    for word_id in indexed_tokens:
        segments_ids.append(seq_id)  # seq_id=o or 1을 추가

        # [SEP]를 발견하면 2번째 문장이 되므로 이후 id를 1로
        if word_id == 102:  # ID 102가 [SEP]이다
            seq_id = 1

    return segments_ids


# 실행
segments_ids = seq2id(indexed_tokens)
print(segments_ids)


[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [ ]:
# 모델로 추론

# 리스트를 PyTorch의 텐서로 하여 모델에 입력
tokens_tensor = torch.tensor([indexed_tokens])
segments_tensors = torch.tensor([segments_ids])

# 추론
with torch.no_grad():
    prediction_scores, seq_relationship_score = net(
        tokens_tensor, segments_tensors)

# 추론한 ID를 단어로 되돌리기
predicted_index = torch.argmax(prediction_scores[0, masked_index]).item()
predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])[0]
print(predicted_token)


bank


## 사전 학습 과제 Next Sentence Prediction를 시험하기

In [ ]:
print(seq_relationship_score)
print(torch.sigmoid(seq_relationship_score))


In [ ]:
# 위 출력을 보면, 클래스0: 두 문장이 의미를 가지는 연속한 것, 클래스1: 두 문장은 무관계
# 중에서, 클래스1의 무관계로 판정하고 있음.

# 입력 문장은
# text = "[CLS] I accessed the bank account. [SEP] We play soccer at the bank of the river. [SEP]"
# 와 무관계였으므로, 제대로 Next Sentence Prediction가 동작했음을 알 수 있다.

끝